# 导入库

In [1]:
from keras.layers import Input,LSTM,Dense
from keras.models import Model,load_model
from keras.utils import plot_model

import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin'
#我的graphviz环境没配好，为了后面的Plot_model

In [3]:
def create_model(n_input,n_output,n_units):
    #训练阶段
    #encoder
    encoder_input = Input(shape = (None, n_input))
    #encoder输入维度n_input为每个时间步的输入xt的维度，这里是用来one-hot的英文字符数
    encoder = LSTM(n_units, return_state=True)
    #n_units为LSTM单元中每个门的神经元的个数，return_state设为True时才会返回最后时刻的状态h,c
    _,encoder_h,encoder_c = encoder(encoder_input)
    encoder_state = [encoder_h,encoder_c]
    #保留下来encoder的末状态作为decoder的初始状态
    
    #decoder
    decoder_input = Input(shape = (None, n_output))
    #decoder的输入维度为中文字符数
    decoder = LSTM(n_units,return_sequences=True, return_state=True)
    #训练模型时需要decoder的输出序列来与结果对比优化，故return_sequences也要设为True
    decoder_output, _, _ = decoder(decoder_input,initial_state=encoder_state)
    #在训练阶段只需要用到decoder的输出序列，不需要用最终状态h.c
    decoder_dense = Dense(n_output,activation='softmax')
    decoder_output = decoder_dense(decoder_output)
    #输出序列经过全连接层得到结果
    
    #生成的训练模型
    model = Model([encoder_input,decoder_input],decoder_output)
    #第一个参数为训练模型的输入，包含了encoder和decoder的输入，第二个参数为模型的输出，包含了decoder的输出
    
    #推理阶段，用于预测过程
    #推断模型—encoder
    encoder_infer = Model(encoder_input,encoder_state)
    
    #推断模型-decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))    
    decoder_state_input = [decoder_state_input_h, decoder_state_input_c]#上个时刻的状态h,c   
    
    decoder_infer_output, decoder_infer_state_h, decoder_infer_state_c = decoder(decoder_input,initial_state=decoder_state_input)
    decoder_infer_state = [decoder_infer_state_h, decoder_infer_state_c]#当前时刻得到的状态
    decoder_infer_output = decoder_dense(decoder_infer_output)#当前时刻的输出
    decoder_infer = Model([decoder_input]+decoder_state_input,[decoder_infer_output]+decoder_infer_state)
    
    return model, encoder_infer, decoder_infer

In [4]:
N_UNITS = 256
BATCH_SIZE = 64
EPOCH = 200
NUM_SAMPLES = 10000

# 数据处理

In [5]:
data_path = 'data/cmn.txt'

## 读取数据

In [6]:
df = pd.read_table(data_path,header=None).iloc[:NUM_SAMPLES,:,]
df.columns=['inputs','targets']

df['targets'] = df['targets'].apply(lambda x: '\t'+x+'\n')

input_texts = df.inputs.values.tolist()
target_texts = df.targets.values.tolist()

input_characters = sorted(list(set(df.inputs.unique().sum())))
target_characters = sorted(list(set(df.targets.unique().sum())))

In [7]:
INUPT_LENGTH = max([len(i) for i in input_texts])
OUTPUT_LENGTH = max([len(i) for i in target_texts])
INPUT_FEATURE_LENGTH = len(input_characters)
OUTPUT_FEATURE_LENGTH = len(target_characters)

## 向量化

In [8]:
encoder_input = np.zeros((NUM_SAMPLES,INUPT_LENGTH,INPUT_FEATURE_LENGTH))
decoder_input = np.zeros((NUM_SAMPLES,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH))
decoder_output = np.zeros((NUM_SAMPLES,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH))

In [9]:
input_dict = {char:index for index,char in enumerate(input_characters)}
input_dict_reverse = {index:char for index,char in enumerate(input_characters)}
target_dict = {char:index for index,char in enumerate(target_characters)}
target_dict_reverse = {index:char for index,char in enumerate(target_characters)}

In [10]:
for seq_index,seq in enumerate(input_texts):
    for char_index, char in enumerate(seq):
        encoder_input[seq_index,char_index,input_dict[char]] = 1

In [11]:
for seq_index,seq in enumerate(target_texts):
    for char_index,char in enumerate(seq):
        decoder_input[seq_index,char_index,target_dict[char]] = 1.0
        if char_index > 0:
            decoder_output[seq_index,char_index-1,target_dict[char]] = 1.0

## 观察向量化的数据

In [12]:
''.join([input_dict_reverse[np.argmax(i)] for i in encoder_input[0] if max(i) !=0])

'Hi.'

In [13]:
''.join([target_dict_reverse[np.argmax(i)] for i in decoder_output[0] if max(i) !=0])

'嗨。\n'

In [14]:
''.join([target_dict_reverse[np.argmax(i)] for i in decoder_input[0] if max(i) !=0])

'\t嗨。\n'

# 创建模型

In [15]:
model_train, encoder_infer, decoder_infer = create_model(INPUT_FEATURE_LENGTH, OUTPUT_FEATURE_LENGTH, N_UNITS)

In [16]:
#查看模型结构
plot_model(to_file='model.png',model=model_train,show_shapes=True)
plot_model(to_file='encoder.png',model=encoder_infer,show_shapes=True)
plot_model(to_file='decoder.png',model=decoder_infer,show_shapes=True)

In [17]:
model_train.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [18]:
model_train.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 73)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 2623)   0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 337920      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  2949120     input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [19]:
encoder_infer.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 73)          0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 256), (None, 256) 337920    
Total params: 337,920
Trainable params: 337,920
Non-trainable params: 0
_________________________________________________________________


In [20]:
decoder_infer.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, 2623)   0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  2949120     input_2[0][0]                    
                                                                 input_3[0][0]                    
          

# 模型训练

In [21]:
model_train.fit([encoder_input,decoder_input],decoder_output,batch_size=BATCH_SIZE,epochs=EPOCH,validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/200
8000/8000 [==============================] - 154s 19ms/step - loss: 2.0315 - val_loss: 2.4918
Epoch 2/200
8000/8000 [==============================] - 143s 18ms/step - loss: 1.9013 - val_loss: 2.4023
Epoch 3/200
8000/8000 [==============================] - 144s 18ms/step - loss: 1.8018 - val_loss: 2.3162
Epoch 4/200
8000/8000 [==============================] - 148s 18ms/step - loss: 1.7133 - val_loss: 2.2307
Epoch 5/200
8000/8000 [==============================] - 156s 20ms/step - loss: 1.6322 - val_loss: 2.1506
Epoch 6/200
8000/8000 [==============================] - 144s 18ms/step - loss: 1.5603 - val_loss: 2.0892
Epoch 7/200
8000/8000 [==============================] - 146s 18ms/step - loss: 1.4961 - val_loss: 2.0370
Epoch 8/200
8000/8000 [==============================] - 150s 19ms/step - loss: 1.4367 - val_loss: 1.9959
Epoch 9/200
8000/8000 [==============================] - 143s 18ms/step - loss: 1.3825 - val_loss: 1.956

8000/8000 [==============================] - 137s 17ms/step - loss: 0.0234 - val_loss: 2.5648
Epoch 154/200
8000/8000 [==============================] - 136s 17ms/step - loss: 0.0224 - val_loss: 2.5735
Epoch 155/200
8000/8000 [==============================] - 136s 17ms/step - loss: 0.0219 - val_loss: 2.5742
Epoch 156/200
8000/8000 [==============================] - 136s 17ms/step - loss: 0.0216 - val_loss: 2.5764
Epoch 157/200
8000/8000 [==============================] - 138s 17ms/step - loss: 0.0212 - val_loss: 2.5900
Epoch 158/200
8000/8000 [==============================] - 137s 17ms/step - loss: 0.0208 - val_loss: 2.5951
Epoch 159/200
8000/8000 [==============================] - 136s 17ms/step - loss: 0.0204 - val_loss: 2.5881
Epoch 160/200
8000/8000 [==============================] - 138s 17ms/step - loss: 0.0201 - val_loss: 2.5881
Epoch 161/200
8000/8000 [==============================] - 135s 17ms/step - loss: 0.0199 - val_loss: 2.6009
Epoch 162/200
8000/8000 [=================

# 预测序列

In [22]:
def predict_chinese(source,encoder_inference, decoder_inference, n_steps, features):
    #先通过推理encoder获得预测输入序列的隐状态
    state = encoder_inference.predict(source)
    #第一个字符'\t',为起始标志
    predict_seq = np.zeros((1,1,features))
    predict_seq[0,0,target_dict['\t']] = 1

    output = ''
    #开始对encoder获得的隐状态进行推理
    #每次循环用上次预测的字符作为输入来预测下一次的字符，直到预测出了终止符
    for i in range(n_steps):#n_steps为句子最大长度
        #给decoder输入上一个时刻的h,c隐状态，以及上一次的预测字符predict_seq
        yhat,h,c = decoder_inference.predict([predict_seq]+state)
        #注意，这里的yhat为Dense之后输出的结果，因此与h不同
        char_index = np.argmax(yhat[0,-1,:])
        char = target_dict_reverse[char_index]
        output += char
        state = [h,c]#本次状态做为下一次的初始状态继续传递
        predict_seq = np.zeros((1,1,features))
        predict_seq[0,0,char_index] = 1
        if char == '\n':#预测到了终止符则停下来
            break
    return output

In [23]:
for i in range(1000,1100):
    test = encoder_input[i:i+1,:,:]#i:i+1保持数组是三维
    out = predict_chinese(test,encoder_infer,decoder_infer,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH)
    #print(input_texts[i],'\n---\n',target_texts[i],'\n---\n',out)
    print(input_texts[i])
    print(out)

I have brothers.
我有兄弟。

I have ten pens.
我有十支筆。

I have to hurry!
我要赶紧了!

I have two cats.
我有两只猫。

I have two sons.
我有兩個兒子。

I just threw up.
我剛才吐了。

I lent him a CD.
我借给他一盘CD。

I like Tom, too.
我也喜歡湯姆。

I like football.
我喜歡足球。

I like potatoes.
我喜歡土豆。

I like the cold.
我喜歡寒冷。

I like this dog.
我喜欢这只狗。

I like your car.
我喜欢您的车。

I lived in Rome.
我住在羅馬。

I love this car.
我愛這台車。

I might say yes.
我可能会说是。

I must help her.
我必須幫助她。

I need a friend.
我需要个朋友。

I need evidence.
我需要證據。

I need you here.
我需要你在這裡。

I paid the bill.
我买了单。

I played tennis.
我打網球了。

I run every day.
我每天跑步。

I speak Swedish.
我说瑞典语。

I talked to her.
我跟她谈了话。

I teach Chinese.
我教中文。

I think it's OK.
我想沒關係。

I took a shower.
我洗了澡。

I want a guitar.
我想要一把吉他。

I want that bag.
我想要那個袋子。

I want to drive.
我想開車。

I was surprised.
我吃惊了。

I wish you'd go.
我希望你去。

I woke up early.
我起得早。

I work too much.
我工作得太多了。

I'll bring wine.
我会带酒来。

I'll never stop.
我絕不會停。

I'm a foreigner.
我是一個外國人。

I'm a night owl.
我是個夜貓子。

I'm about 